In [1]:
import pandas as pd
import re

def data_1C(pth1, pth2, pth3, pth4):
    df1 = pd.read_excel(pth1, dtype=str)
    df1 = df1.drop(index=0)

    df1['тип'] = ['Колобокс_Шины' for i in range(len(df1))]
    df1 = df1.rename(columns={'Ширина': 'Шир', 'Диаметр': 'Диам', 'Профиль': 'Проф', 'Марка (бренд)': 'Марка'})

    df2 = pd.read_excel(pth2, dtype=str)
    df2['тип'] = ['легковая_резина' for i in range(len(df2))]

    df3 = pd.read_excel(pth3, dtype=str)
    df3['тип'] = ['грузовая_резина' for i in range(len(df3))]

    df4 = pd.read_excel(pth4, dtype=str)
    df4['тип'] = ['диски' for i in range(len(df4))]

    return df1, df2, df3, df4

def concat_and_clear(lst):
    df = pd.concat(lst, ignore_index=True)
    rm = ['Сезонность', 
        'Артикул', 
        'Код 1С', 
        'Код_произв',  
        'РРЦ', 
        'Тип_диска', 
        'Отв', 
        'PCD', 
        'DPCD', 
        'Вылет', 
        'Ступица',
        'RN', 
        'Остаток', 
        'Резерв', 
        'Заказ', 
        'Цена_руб',
        'Распр', 
        'Происх', 
        'Сезон',
        'Скор',
        'Размер',
        'Шип']

    df = df.drop(columns=rm)
    df = df.rename(columns={'Шир': 'Ширина', 'Диам': 'Радиус', 'Проф': 'Высота', 'Модель': 'Линейка'})

    df = df[df['Ширина']!="@ширина рез."]
    df = df[df['Высота']!="@профиль"]
    df = df[df['Радиус']!="@диаметр"]
    df = df[df['Линейка']!="@модель дис."]
    df = df.reset_index(drop=True)

    return df

pth1 = "/home/sondors/Documents/price/BERT_NER/csv/Прайс_Колобокс_Шины_2018-07-07 (XLS).xls"
pth2 = "/home/sondors/Documents/price/BERT_NER/csv/легковая_резина.xls"
pth3 = "/home/sondors/Documents/price/BERT_NER/csv/грузовая_резина.xls"
pth4 = "/home/sondors/Documents/price/BERT_NER/csv/диски.xls"

df1, df2, df3, df4 = data_1C(pth1, pth2, pth3, pth4)
df_opensource = concat_and_clear([df1, df2, df3, df4])
df_opensource

,Ширина,Высота,Радиус,Марка,Наименование,тип,Линейка
0,175,65,14,Amtel,Amtel 175/65R14 82Q Nordmaster (В-228),Колобокс_Шины,NaN
1,175,70,13,Avatyre,Avatyre 175/70R13 82Q Freeze,Колобокс_Шины,NaN
2,205,55,16,Avatyre,Avatyre 205/55R16 91T Freeze,Колобокс_Шины,NaN
3,215,65,16,Avatyre,Avatyre 215/65R16 98T Freeze,Колобокс_Шины,NaN
4,215,65,16,Avatyre,Avatyre 215/65R16 98T Freeze Товар с уценкой,Колобокс_Шины,NaN
...,...,...,...,...,...,...,...
7013,"5,5",NaN,14,СКАД,"СКАД Эверест 5,5\R14 4*100 ET38 d67,1 [083...",диски,Эверест
7014,"7,0",NaN,17,СКАД,"СКАД Эссен 7,0\R17 5*100 ET46 d56,1 Алмаз ...",диски,Эссен
7015,"7,0",NaN,17,СКАД,"СКАД Эссен 7,0\R17 5*100 ET46 d56,1 [2850508]",диски,Эссен
7016,"7,0",NaN,17,СКАД,"СКАД Эссен 7,0\R17 5*108 ET50 d63,35 Алмаз...",диски,Эссен


In [2]:
# Функция для проверки, содержит ли значение в "Наименование" значение из указанной колонки и округления этой колонки (если не NaN) и исправление . и ,
import numpy as np

def process_column(row, column_name):
    if not pd.isna(row[column_name]):
        contains_column_value = str(row['Наименование']).find(str(row[column_name])) != -1
        if not contains_column_value:
            row[column_name] = row[column_name].replace('.', ',')
    return row

def process_brand_line(row, column_name):
    if not pd.isna(row[column_name]):
        contains_column_value = str(row['Наименование']).find(str(row[column_name])) != -1
        if not contains_column_value:
            row[column_name] =  np.nan
    return row

columns_to_process = ['Ширина', 'Высота', 'Радиус']

for column in columns_to_process:
    df_opensource = df_opensource.apply(process_column, args=(column,), axis=1)

columns_to_process = ['Марка', 'Линейка']
for column in columns_to_process:
    df_opensource = df_opensource.apply(process_brand_line, args=(column,), axis=1)

df_opensource

,Ширина,Высота,Радиус,Марка,Наименование,тип,Линейка
0,175,65,14,Amtel,Amtel 175/65R14 82Q Nordmaster (В-228),Колобокс_Шины,NaN
1,175,70,13,Avatyre,Avatyre 175/70R13 82Q Freeze,Колобокс_Шины,NaN
2,205,55,16,Avatyre,Avatyre 205/55R16 91T Freeze,Колобокс_Шины,NaN
3,215,65,16,Avatyre,Avatyre 215/65R16 98T Freeze,Колобокс_Шины,NaN
4,215,65,16,Avatyre,Avatyre 215/65R16 98T Freeze Товар с уценкой,Колобокс_Шины,NaN
...,...,...,...,...,...,...,...
7013,"5,5",NaN,14,СКАД,"СКАД Эверест 5,5\R14 4*100 ET38 d67,1 [083...",диски,Эверест
7014,"7,0",NaN,17,СКАД,"СКАД Эссен 7,0\R17 5*100 ET46 d56,1 Алмаз ...",диски,Эссен
7015,"7,0",NaN,17,СКАД,"СКАД Эссен 7,0\R17 5*100 ET46 d56,1 [2850508]",диски,Эссен
7016,"7,0",NaN,17,СКАД,"СКАД Эссен 7,0\R17 5*108 ET50 d63,35 Алмаз...",диски,Эссен


In [3]:
df_opensource = df_opensource.rename(columns={'Ширина': 'width', 'Радиус': 'radius', 'Высота': 'height', 'Марка': 'brand', 'Наименование': 'offer', 'Линейка': 'line', 'тип': 'type'})
df_opensource

,width,height,radius,brand,offer,type,line
0,175,65,14,Amtel,Amtel 175/65R14 82Q Nordmaster (В-228),Колобокс_Шины,NaN
1,175,70,13,Avatyre,Avatyre 175/70R13 82Q Freeze,Колобокс_Шины,NaN
2,205,55,16,Avatyre,Avatyre 205/55R16 91T Freeze,Колобокс_Шины,NaN
3,215,65,16,Avatyre,Avatyre 215/65R16 98T Freeze,Колобокс_Шины,NaN
4,215,65,16,Avatyre,Avatyre 215/65R16 98T Freeze Товар с уценкой,Колобокс_Шины,NaN
...,...,...,...,...,...,...,...
7013,"5,5",NaN,14,СКАД,"СКАД Эверест 5,5\R14 4*100 ET38 d67,1 [083...",диски,Эверест
7014,"7,0",NaN,17,СКАД,"СКАД Эссен 7,0\R17 5*100 ET46 d56,1 Алмаз ...",диски,Эссен
7015,"7,0",NaN,17,СКАД,"СКАД Эссен 7,0\R17 5*100 ET46 d56,1 [2850508]",диски,Эссен
7016,"7,0",NaN,17,СКАД,"СКАД Эссен 7,0\R17 5*108 ET50 d63,35 Алмаз...",диски,Эссен


In [4]:
def find_indices(row):
    indices = {}
    for column in ["width", "height", "radius", "brand", "line"]:
        if not pd.isna(row[column]):
            value = str(row[column])
            start_index = row["offer"].find(value)
            end_index = start_index + len(value) - 1
            indices[column] = (start_index, end_index)
    return indices

indices_df = df_opensource.apply(find_indices, axis=1).apply(pd.Series)
indices_df.columns = [f"{col}_indices" for col in indices_df.columns]
df = pd.concat([df_opensource, indices_df], axis=1)
df

,width,height,radius,brand,offer,type,line,width_indices,height_indices,radius_indices,brand_indices,line_indices
0,175,65,14,Amtel,Amtel 175/65R14 82Q Nordmaster (В-228),Колобокс_Шины,NaN,"(6, 8)","(10, 11)","(13, 14)","(0, 4)",NaN
1,175,70,13,Avatyre,Avatyre 175/70R13 82Q Freeze,Колобокс_Шины,NaN,"(8, 10)","(12, 13)","(15, 16)","(0, 6)",NaN
2,205,55,16,Avatyre,Avatyre 205/55R16 91T Freeze,Колобокс_Шины,NaN,"(8, 10)","(12, 13)","(15, 16)","(0, 6)",NaN
3,215,65,16,Avatyre,Avatyre 215/65R16 98T Freeze,Колобокс_Шины,NaN,"(8, 10)","(12, 13)","(15, 16)","(0, 6)",NaN
4,215,65,16,Avatyre,Avatyre 215/65R16 98T Freeze Товар с уценкой,Колобокс_Шины,NaN,"(8, 10)","(12, 13)","(15, 16)","(0, 6)",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
7013,"5,5",NaN,14,СКАД,"СКАД Эверест 5,5\R14 4*100 ET38 d67,1 [083...",диски,Эверест,"(15, 17)",NaN,"(20, 21)","(0, 3)","(6, 12)"
7014,"7,0",NaN,17,СКАД,"СКАД Эссен 7,0\R17 5*100 ET46 d56,1 Алмаз ...",диски,Эссен,"(13, 15)",NaN,"(18, 19)","(0, 3)","(6, 10)"
7015,"7,0",NaN,17,СКАД,"СКАД Эссен 7,0\R17 5*100 ET46 d56,1 [2850508]",диски,Эссен,"(13, 15)",NaN,"(18, 19)","(0, 3)","(6, 10)"
7016,"7,0",NaN,17,СКАД,"СКАД Эссен 7,0\R17 5*108 ET50 d63,35 Алмаз...",диски,Эссен,"(13, 15)",NaN,"(18, 19)","(0, 3)","(6, 10)"


In [12]:
dict(df.brand.value_counts())

{'Nokian': 683,
 'Нокиан': 352,
 'Goodyear': 318,
 'Maxxis': 287,
 'Бриджстоун': 282,
 "X'trike": 270,
 'Hankook': 268,
 'Данлоп': 257,
 'Bridgestone': 253,
 'Мишелин': 245,
 'Pirelli': 237,
 'Michelin': 207,
 'Kumho': 192,
 'Continental': 190,
 'Нордман': 176,
 'КиК': 171,
 'СКАД': 169,
 'Кумхо': 165,
 'Sailun': 140,
 'Cordiant': 133,
 'Laufenn': 121,
 'Пирелли': 106,
 'Nitto': 106,
 'Роудстоун': 103,
 'RepliKey': 98,
 'Yokohama': 97,
 'Тайгер': 88,
 'TREBL': 86,
 'Matador': 70,
 'Gislaved': 65,
 'LANDSAIL': 61,
 'БФ гудрич': 54,
 'Formula': 46,
 'Kama': 46,
 'Sava': 44,
 'Формула': 37,
 'Dunlop': 35,
 'Файрстоун': 34,
 'BFGoodrich': 31,
 'Амт': 31,
 'Toyo': 31,
 'Йокохама': 26,
 'RW Classic': 26,
 'ГУД-ЕАР': 24,
 'НК.ШЗ': 24,
 'Tunga': 22,
 'Яр. ШЗ': 18,
 'Tigar': 18,
 'Replica FR': 17,
 'HIFLY': 15,
 'SATOYA': 15,
 'Корморан': 13,
 'Nordman': 13,
 'Contyre': 13,
 'Сава': 9,
 'MOMO': 9,
 'Волж.ШЗ': 9,
 'Ласса': 9,
 'Аеолус': 8,
 'Ориум': 8,
 'Kormoran': 8,
 'ГАЗ': 5,
 'Avatyre': 5,
 

In [9]:
df[df.radius == "16C"]

,width,height,radius,brand,offer,type,line,width_indices,height_indices,radius_indices,brand_indices,line_indices
138,215,65,16C,Bridgestone,Bridgestone 215/65R16C 109/107R Blizzak W995,Колобокс_Шины,NaN,"(12, 14)","(16, 17)","(19, 21)","(0, 10)",NaN
199,235,65,16C,Bridgestone,Bridgestone 235/65R16C 115/113R Blizzak W995,Колобокс_Шины,NaN,"(12, 14)","(16, 17)","(19, 21)","(0, 10)",NaN
310,195,75,16C,Continental,Continental 195/75R16C 107/105R VancoIceContac...,Колобокс_Шины,NaN,"(12, 14)","(16, 17)","(19, 21)","(0, 10)",NaN
397,235,65,16C,Continental,Continental 235/65R16C 121/119N VancoIceContac...,Колобокс_Шины,NaN,"(12, 14)","(16, 17)","(19, 21)","(0, 10)",NaN
398,235,65,16C,Continental,Continental 235/65R16C 121/119R ContiVanContac...,Колобокс_Шины,NaN,"(12, 14)","(16, 17)","(19, 21)","(0, 10)",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3543,205,65,16C,Tigar,Tigar 205/65R16C 107/105R Cargo Speed Winter,Колобокс_Шины,NaN,"(6, 8)","(10, 11)","(13, 15)","(0, 4)",NaN
3544,205,75,16C,Tigar,Tigar 205/75R16C 110/108R Cargo Speed Winter,Колобокс_Шины,NaN,"(6, 8)","(10, 11)","(13, 15)","(0, 4)",NaN
3545,215,65,16C,Tigar,Tigar 215/65R16C 109/107R Cargo Speed Winter,Колобокс_Шины,NaN,"(6, 8)","(10, 11)","(13, 15)","(0, 4)",NaN
3546,215,75,16C,Tigar,Tigar 215/75R16C 113/111R Cargo Speed Winter,Колобокс_Шины,NaN,"(6, 8)","(10, 11)","(13, 15)","(0, 4)",NaN


# Для запроса

In [23]:
ind = 0
data = f'"offer":"{df.iloc[ind].offer}", "width":{df.iloc[ind].width}, "height":{df.iloc[ind].height}, "radius":{df.iloc[ind].radius}, "brand":"{df.iloc[ind].brand}", "line":"{df.iloc[ind].line}", "width_indices":{df.iloc[ind].width_indices}, "height_indices":{df.iloc[ind].height_indices}, "radius_indices":{df.iloc[ind].radius_indices}, "brand_indices":{df.iloc[ind].brand_indices}, "line_indices":{df.iloc[ind].line_indices}'
data

'"offer":"Amtel 175 / 65 R 14 82Q Nordmaster (В-228)", "width":175, "height":65, "radius":14, "brand":"Amtel", "line":"nan", "width_indices":(6, 8), "height_indices":(12, 13), "radius_indices":(17, 18), "brand_indices":(0, 4), "line_indices":nan'

In [ ]:
"""
Пример первой строки из таблицы:
data = {"offer":"Amtel 175 / 65 R 14 82Q Nordmaster (В-228)", "width":175, "height":65, "radius":14, "brand":"Amtel", "width_indices":(6, 8), "height_indices":(12, 13), "radius_indices":(17, 18), "brand_indices":(0, 4)}
df = pd.DataFrame(data, index=[0])

"""

In [9]:
dict(df_csv['BIO_Tags'].value_counts())

{'B-brand B-width O B-height O B-radius O O O O': 1574,
 'B-brand B-width O B-height O B-radius O O O O O': 1272,
 'B-brand B-width O B-height O B-radius O O O O O O': 860,
 'B-brand B-width O B-height O B-radius O O O': 663,
 'B-brand B-width O B-height O B-radius O O O O O O O': 525,
 'B-brand B-width O B-height O B-radius O O B-line': 328,
 'B-brand B-line B-width O O B-radius O O O O O': 241,
 'B-brand B-width O B-height O B-radius O O': 134,
 'B-brand B-line B-width O O B-radius O O O O O O O': 117,
 'B-brand B-width O B-height O B-radius O O O O O O O O': 104,
 'B-brand B-line B-width O O B-radius O O O O': 77,
 'O O O B-width O B-height O B-radius O': 77,
 'B-brand O B-line B-width O O B-radius O O O O O': 70,
 'B-brand O O B-width O O B-radius O O O O O': 50,
 'B-brand B-width O B-height O B-radius O O O O O B-line': 43,
 'B-brand O O B-width O O B-radius O O O O O O': 41,
 'O O O O B-width O B-height O B-radius O': 35,
 'B-brand O O B-line B-width O O B-radius O O O O O': 30,


In [12]:
df_csv[df_csv['BIO_Tags'] == 'O O O B-width O B-height O B-radius']

,width,height,radius,brand,name,type,line,BIO_Tags
3827,175,65.0,14,НК.ШЗ,Viatti Brina V-521 175 / 65 / 14,легковая_резина,Brina V-521,O O O B-width O B-height O B-radius
3828,185,60.0,15,НК.ШЗ,Viatti Brina V-521 185 / 60 / 15,легковая_резина,Brina V-521,O O O B-width O B-height O B-radius
3829,185,65.0,14,НК.ШЗ,Viatti Brina V-521 185 / 65 / 14,легковая_резина,Brina V-521,O O O B-width O B-height O B-radius
3831,195,60.0,15,НК.ШЗ,Viatti Brina V-521 195 / 60 / 15,легковая_резина,Brina V-521,O O O B-width O B-height O B-radius
3835,215,50.0,17,НК.ШЗ,Viatti Brina V-521 215 / 50 / 17,легковая_резина,Brina V-521,O O O B-width O B-height O B-radius
3836,215,55.0,16,НК.ШЗ,Viatti Brina V-521 215 / 55 / 16,легковая_резина,Brina V-521,O O O B-width O B-height O B-radius
4505,175,65.0,14,НК.ШЗ,КАМА BREEZE НК-132 175 / 65 / 14,легковая_резина,КАМА BREEZE НК-132,O O O B-width O B-height O B-radius
4506,185,70.0,14,НК.ШЗ,КАМА BREEZE НК-132 185 / 70 / 14,легковая_резина,КАМА BREEZE НК-132,O O O B-width O B-height O B-radius
4554,205,70.0,15,Срш,Кордиант ALL TERRAIN 205 / 70 / 15,легковая_резина,ALL TERRAIN,O O O B-width O B-height O B-radius
4555,215,65.0,16,Срш,Кордиант ALL TERRAIN 215 / 65 / 16,легковая_резина,ALL TERRAIN,O O O B-width O B-height O B-radius
